In [ ]:
library(tidyverse)
library(repr)
library(readxl)
library(infer)
library(cowplot)
library(GGally)
library(broom)
library(dplyr)
library(AER)
library(digest)
library(gridExtra)
library(caret)
library(pROC)
library(boot)
library(glmnet)
library(leaps)
library(faraway)
library(mltools)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.5 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.3      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Registered S3 method overwritten by 'GGally':
  method from   
  +.gg   ggplot2

Loading required package: car

Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:dplyr’:

    recode


The following object is masked from ‘package:purrr’:

    some


Loading required package: lmtest

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Loading required package: sandwich

Loading required package: survival


Attaching package: ‘gridExtra’


The following obj

In [ ]:
set.seed(4)
crime_data <- read_csv("data/communities.data", col_names = FALSE)

head(crime_data)

In [ ]:
crime <- crime_data %>%
  select(X1,X6,X14,X17,X26,X34,X36,X38,X47,X48,X128)%>%
  rename(state = X1,
         popComm = X6,
         agePct16t24 = X14,
         pctUrban = X17,
         perCapInc = X26,
         pctUnderPov = X34,
         pctNotHSGrad = X36,
         pctUnemployed = X38,
         pctDiv = X47,
         meanPerFam = X48,
         totCrimesPerPop = X128)
crime$pctUrban <- if_else(crime$pctUrban > 0,
                 "urban","non-urban")
crime <- crime%>%
  mutate(pctUrban = as_factor(pctUrban))

head(crime)
tail(crime)
nrow(crime)

In [ ]:
clean_crime <- crime %>%
filter(state == 6) %>%
select(- state)
head(clean_crime)

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 4)

tot_crime_dist <- clean_crime %>%
    ggplot(aes(x = totCrimesPerPop)) +
    geom_histogram(bins = 30, color = "white") +
    xlab("Total number of violent crimes per 100K population") +
    ylab("Count") +
    ggtitle("Distribution of total crimes per 100K population") +
    theme(text = element_text(size = 10)) +
    theme(plot.title = element_text(hjust = 0.45))
tot_crime_dist

In [ ]:
# options(repr.plot.width = 11, repr.plot.height = 6)

# crime_pair_plots <- clean_crime %>%
#     ggpairs(progress = FALSE) +
#   theme(
#     text = element_text(size = 10),
#     plot.title = element_text(face = "bold"),
#     axis.title = element_text(face = "bold"),
#     axis.text.x = element_text(angle = 20, hjust = 1)
# )
# crime_pair_plots

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 4)

urban_totCrime_boxplot <- 
    clean_crime %>%
    ggplot(aes(x = pctUrban, y = totCrimesPerPop , color = pctUrban)) +
    geom_boxplot() +
    xlab("Urbanization") +
    ylab("Total number of violent crimes per 100K population") +
    ggtitle("Total number of violent crimes per 100K population for Urban and Non-urban areas") +
    theme(text = element_text(size = 10)) +
    theme(plot.title = element_text(hjust = 0.5))

urban_totCrime_boxplot

In [ ]:
crime_summary <-
    clean_crime %>%
    select(-pctUrban) %>%
    pivot_longer(cols = everything()) %>%
    group_by(name) %>% 
    summarise(
    mean = mean(value,na.rm = T),
    sd = sd(value,na.rm = T),
    median = median(value, na.rm = T),
    variance = var(value, na.rm = T),
    max = max(value, na.rm = T),
    min = min(value, na.rm = T))

crime_summary

**Check Model Assumption**

In [ ]:
#check heteroscedastic
model <- lm(formula = log(totCrimesPerPop) ~., data = clean_crime)

options(repr.plot.width = 5.5, repr.plot.height = 5.5)
plot(model, 1, main = "Checking whether heteroscedastic")


In [ ]:
#check normality 
plot(model, 2, main = "Model")

hist(residuals(object = model),
  breaks = 10,
  main = "Histogram of Residuals for Model",
  xlab = "Residuals"
)

In [ ]:
#check multicollinearity
corr_matrix_crime <- 
    clean_crime %>%
    select(- pctUrban) %>%
    cor() %>%
    as.data.frame() %>%
    rownames_to_column("var1") %>%
    pivot_longer(- var1, names_to = "var2", values_to = "corr")
head(corr_matrix_crime)

In [ ]:
# plot vif
options(repr.plot.width = 7, repr.plot.height = 7)

plot_corr_matrix_crime <- corr_matrix_crime %>%
  ggplot(aes(var1, var2)) +
  geom_tile(aes(fill = corr), color = "white") +
  scale_fill_distiller("Correlation Coefficient \n",
    palette =  "YlOrRd",
    direction = 1, limits = c(-1,1)
  ) +
  labs(x = "Variable 1", y = "Variable 2") +
  theme_minimal() +
  theme(
    axis.text.x = element_text(
      angle = 45, vjust = 1,
      size = 8, hjust = 1
    ),
    axis.text.y = element_text(
      vjust = 1,
      size = 8, hjust = 1
    ),
    legend.title = element_text(size = 8, face = "bold"),
    legend.text = element_text(size = 8),
    legend.key.size = unit(1, "cm")
  ) +
  coord_fixed() +
  geom_text(aes(var1, var2, label = round(corr, 2)), color = "black", size = 3.8)
plot_corr_matrix_crime

In [ ]:
# get vif of variables
VIF_crime <- vif(model)
round(VIF_crime, 3)

In [ ]:
# Divide data into training set and testing set
set.seed(4)

clean_crime$ID <- 1:nrow(clean_crime)
crime_train <- 
    slice_sample(clean_crime, prop = 0.70) 
    
crime_test <- 
    clean_crime %>%
    anti_join(crime_train, by = "ID")

head(crime_train)
nrow(crime_train)
head(crime_test)
nrow(crime_test)

In [ ]:
crime_train <- crime_train %>% select(- "ID")
crime_test <- crime_test %>% select(- "ID")

In [ ]:
# forward selection for predictive
crime_forward_sel <- regsubsets(
    x = totCrimesPerPop ~., nvmax = 9,
    data = crime_train,
    method = "forward")

crime_forward_summary <- summary(crime_forward_sel)
crime_forward_summary

In [ ]:
crime_forward_summary_df <- tibble(
    n_input_variables = 1:9,
    RSQ = crime_forward_summary$rsq,
    RSS = crime_forward_summary$rss,
    ADJ.R2 = crime_forward_summary$adjr2,
    Cp = crime_forward_summary$cp,
    BIC = crime_forward_summary$bic,
)
crime_forward_summary_df
#for inference we look at ADJ.R2(higher is better); for predictive, we look at Cp(lower is better)

In [ ]:
# for inference model, we want to compare a full model and a reduced model

#full model
crime_full_OLS_inf <- lm(formula = totCrimesPerPop ~., data = crime_train)
tidy(crime_full_OLS_inf)

#reduced model
crime_reduced_OLS_inf <- lm(formula = totCrimesPerPop ~ popComm + pctUrban + perCapInc + pctUnderPov + pctNotHSGrad 
                            + pctUnemployed + pctDiv + meanPerFam, data = crime_train)
tidy(crime_reduced_OLS_inf)

In [ ]:
glance(crime_full_OLS_inf)$r.squared
glance(crime_reduced_OLS_inf)$r.squared

In [ ]:
# choose cp
plot(summary(crime_forward_sel)$cp,
  main = "Cp for forward selection",
  xlab = "Number of Input Variables", ylab = "Rsq", type = "b", pch = 9,
  col = "red"
)

In [ ]:
# for predictive model, we also compare a full model and a reduced model

#full model
crime_full_OLS_pred <- lm(formula = totCrimesPerPop ~., data = crime_train)
test_crime_full_OLS_pred <- predict(crime_full_OLS_pred, newdata = crime_test)
crime_R_MSE_pred_models <- tibble(
    Model = "OLS Full Regression",
    R_MSE = rmse(
        preds = test_crime_full_OLS_pred,
        actuals = crime_test$totCrimesPerPop))
crime_R_MSE_pred_models


In [ ]:
# reduced model
crime_reduced_OLS_pred <- lm(formula = totCrimesPerPop ~ popComm + pctUrban + perCapInc + pctUnderPov + pctNotHSGrad 
                            + pctUnemployed + pctDiv + meanPerFam, data = crime_train)
test_crime_reduced_OLS_pred <- predict(crime_reduced_OLS_pred, newdata = crime_test)
crime_R_MSE_pred_models <- rbind(
    crime_R_MSE_pred_models,
    tibble(
        Model = "OLS Reduced Regression",
        R_MSE = rmse(
            preds = test_crime_reduced_OLS_pred,
            actuals = crime_test$totCrimesPerPop)))
crime_R_MSE_pred_models

## Lasso

In [ ]:
# change categorical into 0 and 1

crime_train <- crime_train %>%
    mutate(pctUrban = if_else(pctUrban == "urban", 1, 0))

crime_test <- crime_test %>%
    mutate(pctUrban = if_else(pctUrban == "urban", 1, 0))

head(crime_train)

In [ ]:
# build matrix 
crime_train_X <- crime_train %>% select(- "totCrimesPerPop") %>% as.matrix()
crime_train_Y <- crime_train %>% select("totCrimesPerPop") %>% as.matrix()

crime_test_X <- crime_test %>% select(- "totCrimesPerPop") %>% as.matrix()
crime_test_Y <- crime_test %>% select("totCrimesPerPop") %>% as.matrix()


In [ ]:
# choose lambda
crime_cv_lambda_LASSO <- cv.glmnet(
    x = crime_train_X, 
    y = crime_train_Y,
    alpha = 1)

plot(crime_cv_lambda_LASSO, main = "Lambda selection by CV with LASSO\n\n")

In [ ]:
crime_lambda_min_MSE_LASSO <- round(crime_cv_lambda_LASSO$lambda.min, 4)
crime_lambda_min_MSE_LASSO

In [ ]:
crime_cv_lambda_LASSO <- glmnet(
    x = crime_train_X, 
    y = crime_train_Y,
    alpha = 1,
    lambda = crime_lambda_min_MSE_LASSO)

In [ ]:
crime_test_pred_LASSO_min <- predict(crime_cv_lambda_LASSO, newx = crime_test_X)

In [ ]:
crime_R_MSE_pred_models <- rbind(crime_R_MSE_pred_models,
    tibble(
    Model = "LASSO Regression with minimum MSE",
    R_MSE = rmse(
        preds = crime_test_pred_LASSO_min,
        actuals = crime_test$totCrimesPerPop)))
crime_R_MSE_pred_models